In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from climakitae.core.data_interface import get_data, get_data_options

In [ ]:
wrf_data = get_data(
    variable="Evapotranspiration",
    downscaling_method="Dynamical", 
    resolution="3 km",
    timescale="daily",
    units="mm/day",
    approach="Warming Level",
    warming_level=[1.5],  # Must be a list
    enable_hidden_vars=True,  # CRITICAL: Enable hidden variables
)